In [1]:
from scitran_client import ScitranClient, el
client = ScitranClient('scitran')

Found an existing token for this instance.
The existing token has been refreshed.


In [2]:
# Sessions belong to projects. We can thus directly the 'label' field of 'projects'.
query = el.filter_('projects', el.term('label', 'vwfa'))
sessions = client.search_sessions(query)

# Files belong to acquisitions. We can filter files by their type (nifti, in this
# example), as well as by properties of their acquisitions.
query = el.filter_('files', el.term('type', 'nifti'))
# Acquisitions belong to sessions. We can make sure the acquisitions correspond
# to a session we are interested in and make sure they have a useful type for us.
query.update(el.filter_('acquisitions', el.and_(
    el.term('measurement', 'anatomy_t1w'),
    el.or_(*[el.term('session', session['_id']) for session in sessions]),
)))
files = client.search_files(query)

In [3]:
# Let's analyze the first file.
example_file = files[0]

import os
# fsl-bet looks for files in the nifti subdirectory
nifti_dir = os.path.join(client.gear_in_dir, 'nifti')
if not os.path.exists(nifti_dir):
    os.mkdir(nifti_dir)
example_file_path = client.download_all_file_search_results([example_file], dest_dir=nifti_dir)[0]

100%|██████████| 1/1 [00:00<00:00, 14.25it/s]

Found local copy of 11353_3_1.nii.gz with correct content.


In [4]:
session_id = example_file['_source']['acquisition']['session']

# We let fsl-bet find the input file by having an empty string for a command.
client.run_gear_and_upload_analysis('testing fsl-bet local run', 'scitran/fsl-bet', session_id, '')

Running container scitran/fsl-bet on with input /Users/carlos/Downloads/input and output /Users/carlos/Downloads/output
[scitran/fsl-bet]  Initiated
[scitran/fsl-bet]  Running bet2 on 11353_3_1.nii.gz
[scitran/fsl-bet]  Wrote: 11353_3_1_brain-extracted.nii.gz
[scitran/fsl-bet]  generated /flywheel/v0/output/.metadata.json
[scitran/fsl-bet]  Done!
Docker container finished with exit code 0
Uploading results to collection with id 56e9d386ddea7f915e81f704.
Uploaded analysis has ID 57e4767c766755001727084d. Server responded with 200.


In [6]:
import nibabel as nib
import scipy.misc
img = nib.load(example_file_path)
scipy.misc.toimage(img.get_data()[100, :, :]).save('before.png')

output_file_path = os.path.join(client.gear_out_dir, '11353_3_1_brain-extracted.nii.gz')
img = nib.load(output_file_path)
scipy.misc.toimage(img.get_data()[100, :, :]).save('after.png')

![](before.png) ![](after.png)